<a href="https://colab.research.google.com/github/rodrihgh/music-scales-playground/blob/master/music-scales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/rodrihgh/music-scales-playground/master?filepath=music-scales.ipynb)

# Music scales widget

Imports and utility classes

In [0]:
import warnings
import numpy as np
from scipy import signal
from ipywidgets import Button, IntSlider, Output, Layout, VBox, Dropdown, AppLayout
import matplotlib.pyplot as plt
from IPython.display import Audio, display, clear_output

import sys
in_colab = 'google.colab' in sys.modules

from collections import namedtuple, OrderedDict

%matplotlib inline

continuous_update = False

import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback
        self._task = asyncio.ensure_future(self._job())

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        if continuous_update:
            timer = None
            def debounced(*args, **kwargs):
                nonlocal timer
                def call_it():
                    fn(*args, **kwargs)
                if timer is not None:
                    timer.cancel()
                timer = Timer(wait, call_it)
            return debounced
        else:
            return fn
    return decorator

## Music arithmetic

In [0]:
octave_cents = 1200

fifth_circle_octaves = 7

note_names = ['C', 'C#', 'D', 'D#', 'E',
              'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

roman_nums = ['I', 'II', 'III', 'IV', 'V', 'VI',
              'VII', 'VIII', 'IX', 'X', 'XI', 'XII']

num_halftones = len(note_names)
halftone_cents = round(octave_cents / num_halftones)

a3_hz = 220
a3_index = note_names.index('A')


def ratio2cents(ratio):
  return np.log2(ratio) * octave_cents


def cents2ratio(cents):
  return np.power(2, np.divide(cents, octave_cents))


def octave_fit(cents):
  return cents % octave_cents


def tet_fit(cents):
  return round(cents / halftone_cents) * halftone_cents


def stacked_fifths(num_notes, fifth_val, root):
    
    note_cents = [octave_fit(n * fifth_val) for n in range(num_notes)]

    ref_cents = note_cents[root]
    note_cents = [round(octave_fit(cents - ref_cents)) for cents in note_cents]

    note_cents.sort()
    
    return note_cents
    

fifth_cents = ratio2cents(3 / 2)    # Pure fifth
majthird_cents = ratio2cents(5 / 4) # Pure major third

note_freq_hz = [a3_hz * cents2ratio((n - a3_index) * halftone_cents) for n in range(num_halftones)]

syntcomm_cents = octave_fit(fifth_cents * 4) - majthird_cents # Syntonic comma


# Equal division of the octave
def edo(num_notes):
  tone_step = octave_cents / num_notes
  note_cents = [round(n * tone_step) for n in range(num_notes) ]
  return note_cents



def pythagoras(num_notes=num_halftones):
    
    return stacked_fifths(num_notes, fifth_cents, root=1)
    

def five_limit_just():
    ratios = [1, 16/15, 9/8, 6/5, 5/4, 4/3,
              45/32, 3/2, 8/5, 5/3, 9/5, 15/8]
    note_cents = np.round(ratio2cents(ratios))
    
    return note_cents


def seven_limit_just():
    ratios = [1, 16/15, 9/8, 6/5, 5/4, 4/3,
              7/5, 3/2, 8/5, 5/3, 7/4, 15/8]
    note_cents = np.round(ratio2cents(ratios))
    
    return note_cents


def meantone(comma_split=4):
    mean_tone_fifth = fifth_cents - syntcomm_cents / comma_split
    return stacked_fifths(num_halftones, mean_tone_fifth, root=1)


def twelve_tet():
    note_cents = [round(n * halftone_cents) for n in range(num_halftones)]
    return note_cents


def fifth_order(num_notes, first=0):
    fifths = octave_fit(np.arange(num_notes) * fifth_cents)
    ordered = np.argsort(np.argsort(fifths))
    ordered = (np.append(ordered, 0) + first) % num_notes
    return ordered


def get_note_name(cents, ref):
    
    c_cents = octave_fit(cents + ref)
    c_index = np.round(c_cents / halftone_cents).astype(np.int)
    
    return note_names[c_index % num_halftones]


def get_cents(freq):
    return ratio2cents(freq / note_freq_hz[0])


## Scale and temperament options

In [0]:

pyth_name = "Pure fifths"
edo_name = "Equally divided octave"


# TODO add choice of root note

Temperament = namedtuple('Temperament', ['name', 'get'])
Scale = namedtuple('Temperament', ['name', 'notes'])
Note = namedtuple('Temperament', ['name', 'cents'])


temp_regular = (Temperament("Equal temperament", twelve_tet),
                Temperament(pyth_name, pythagoras),
                Temperament("Just intonation (7-limit)", seven_limit_just),
                Temperament("Meantone (1/4-comma)", meantone),
                Temperament(edo_name, edo))

temp_microtonal = (Temperament(edo_name, edo),
                   Temperament(pyth_name, pythagoras))

scales = OrderedDict()

scales[2] = (Scale("Fifth", (0, 7)),)

scales[3] = (Scale("Structural", (0, 5, 7)),
             Scale("Major triad", (0, 4, 7)),
             Scale("Minor triad", (9, 0, 4)),
             Scale("Diminished triad", (11, 2, 5)),
             Scale("Augmented triad", (5, 9, 1)))

scales[4] = (Scale("Major tetratonic", (0, 2, 5, 7)),
             Scale("Minor tetratonic", (2, 5, 7, 0)),
             Scale("Major seventh", (0, 4, 7, 11)),
             Scale("Minor seventh", (9, 0, 4, 7)),
             Scale("Dominant seventh", (0, 4, 7, 10)),
             Scale("Diminished seventh", (11, 2, 5, 8)),
             Scale("Half-diminished seventh", (11, 2, 5, 9)),
             Scale("Minor-major seventh", (0, 3, 7, 11)))

scales[5] = (Scale("Major pentatonic", (0, 2, 4, 7, 9)),
             Scale("Minor pentatonic", (9, 0, 2, 4, 7)))

scales[6] = (Scale("Major hexatonic", (0, 2, 4, 5, 7, 9)),
             Scale("Minor hexatonic", (2, 4, 5, 7, 9, 0)),
             Scale("Whole-tone", (5, 7, 9, 11, 1, 3)),
             Scale("Blues scale", (9, 0, 2, 3, 4, 7)))

scales[7] = (Scale("Major scale", (0, 2, 4, 5, 7, 9, 11)),
             Scale("Minor scale", (9, 11, 0, 2, 4, 5, 7)),
             Scale("Harmonic minor", (9, 11, 0, 2, 4, 5, 8)),
             Scale("Dorian mode", (2, 4, 5, 7, 9, 11, 0)),
             Scale("Phryigian mode", (4, 5, 7, 9, 11, 0, 2)),
             Scale("Lydian mode", (5, 7, 9, 11, 0, 2, 4)),
             Scale("Mixolyidian mode", (7, 9, 11, 0, 2, 4, 5)),
             Scale("Locrian mode", (11, 0, 2, 4, 5, 7, 9)))

scales[12] = (Scale("Chromatic", tuple(range(num_halftones))),)

scales[17] = (Scale("-", 17),)
scales[24] = (Scale("Quarter-tone", 24),)
scales[53] = (Scale("-", 53),)

valid_num_notes = tuple(scales.keys())


default_num_notes = 7


def get_notes(temperament, notes):
    
    if type(notes) is int:
        tempered_notes = temperament.get(notes)
        
        f_ref = note_freq_hz[0]
        
        note_keys = [str(n+1) for n in range(notes)] if notes > num_halftones else roman_nums
        
        note_tuple = tuple(Note(note_keys[n], tempered_notes[n]) for n in range(notes))
    
        
    elif type(notes) is tuple:
        tempered_notes = temperament.get()

        root = notes[0]
        f_ref = note_freq_hz[root]
        cent_ref = tempered_notes[root]
        
        note_tuple = tuple(Note(note_names[n],
                                octave_fit(tempered_notes[n] - cent_ref)) for n in notes)

    else:
        raise ValueError("notes must be an integer or a tuple of integers.")
        
    return note_tuple, f_ref
    


## Signal processing
My very own synth implementation in less than 200 lines of code!!

In [0]:
sampling_rate = 44100      # Sample rate (Hz)

def time2samples(t, rate):
    """
    Convert time to number of samples at a certain rate.
    :param t: Time in seconds
    :param rate: Sampling rate.
    :return: Number of samples for the given time at the given rate.
    """
    return round(t * rate)


def db2mag(db):
    return 10 ** (db / 20)


def piecewiselin(x, y):
    """
    Create piecewise linear array
    :param x: Lengths of the segments in number of samples.
    :param y: Point values. Its length must be 1 greater than the lenght of x.
    :return: Piecewise linear array.
    """
    xlen = len(x)
    ylen = len(y)

    if (ylen - xlen) != 1:
        raise ValueError(f"Invalid dimensions len(x)={xlen}, len(y)={ylen}. len(x) must equal len(y) - 1.")

    y_start = y[:-1]
    y_stop = y[1:]

    pwl = np.array([])
    for start, stop, num in zip(y_start, y_stop, x):
        pwl = np.append(pwl, np.linspace(start, stop, num))

    return pwl


def adsr(adsr_durations, adsr_levels, total_duration=None, sustain_fade=1, smooth_factor=1):

    n_attack, n_decay, n_sustain, n_release = adsr_durations

    keystroke_duration = n_attack + n_decay + n_sustain + n_release
    
    db_floor, db_peak, db_sustain, db_release = adsr_levels

    if total_duration is not None:
      if total_duration < keystroke_duration:
        raise ValueError("Total duration must be greater or equal than the overall keystroke duration")
        
      fadeout = total_duration - keystroke_duration

      db_slope = (db_floor - db_sustain) / n_release
      db_release += db_slope * fadeout

      n_release += fadeout

    if max(adsr_levels) > -6.0:
        raise warnings.warn(f"Your dB levels are {adsr_levels}. Levels greater than -6 dB may saturate.", UserWarning)
    

    #################################################################
    #                                                               #
    #                       ADSR Curve                              #
    # dB Level ^                                                    #
    #          |                                                    #
    #    Peak  |        X                                           #
    #          |       X XX                                         #
    #          |      X    XX                                       #
    # Sustain  |     X       XXXXXXXXXXXXX                          #
    #          |    X                     XX                        #
    #          |   X                        XX                      #
    #          |  X                           XX                    #
    #          | X                              XX                  #
    #   Floor  |X                                 X                 #
    #          +--------------------------------------------> Time  #
    #          |-----|------------|---------|-------|-------|       #
    #           Attack  Decay      Sustain   Release Fadeout        #
    #################################################################
    
    if smooth_factor > 1:
        
        n_attack //= smooth_factor
        n_decay //= smooth_factor
        n_sustain //= smooth_factor
        n_release //= smooth_factor

    adsr_curve = piecewiselin((n_attack, n_decay, n_sustain, n_release),
                              (db_floor, db_peak, db_sustain,
                               db_sustain - sustain_fade, db_release))
    
    if smooth_factor > 1:
        adsr_curve = signal.resample(adsr_curve, total_duration, window="hamming")
    
    magnitudes = db2mag(adsr_curve)

    return magnitudes


def lowpass_filter(x, fc=8000, order=6, fs=44100):
  """
    Apply Butterworth lowpass-filter to signal
    :param x: Input signal.
    :param fc: Cutoff frequency for the filtering. Default is 8 kHz.
    :param fs: Sampling frequency. Default is 44.1 kHz
    :param order: Order of the butterworth lowpass filter. Default is 4
    :return: Filtered signal
  """

  b, a = signal.butter(order, fc * 2 / fs)
  y = signal.lfilter(b, a, x)

  return y
  

def sawtooth_harmonics(f0, t, n_harmonics=4, power=1, random_phase=False):

  wave = np.zeros_like(t)
  for n in np.arange(1, n_harmonics + 1):
    a = 1 / (n ** power)
    phase = 2 * np.pi * n * f0 * t
    if random_phase:
        phase += np.random.rand() * 2 * np.pi
    wave += a * np.sin(phase)

  return wave


def synthesize(note_frequencies, fref=261.626, fc=1500, fs=44100, fadeout=1,
               attack=.1, decay=.1, sustain=-15, release=.9, sus_time=.2,
               floor=-90, peak=-12, rel_level=-20, spacing=.6, chord_notes=4):
  
  n_freqs = len(note_frequencies)
    
  chord = n_freqs <= chord_notes + 1

  attack_samples = time2samples(attack, fs)
  decay_samples = time2samples(decay, fs)
  sustain_samples = time2samples(sus_time, fs)
  release_samples = time2samples(release, fs)
  fadeout_samples = time2samples(fadeout, fs)

  sample_spacing = time2samples(spacing, fs)
    
  chord_length = sample_spacing + fadeout_samples * 2 if chord else 0  

  total_samples = sample_spacing * n_freqs  + fadeout_samples + chord_length

  adsr_samples = (attack_samples, decay_samples,
                  sustain_samples, release_samples)
  adsr_levels = (floor, peak, sustain, rel_level)
    
  chord_adsr_ivals = (attack_samples, decay_samples,
                      2 * sustain_samples, release_samples)
    
  adsr_curve = adsr(adsr_samples, adsr_levels, total_duration=total_samples)
  adsr_chord = adsr(chord_adsr_ivals, adsr_levels, total_duration=chord_length) if chord else 0
  data = np.zeros(total_samples)

  for n, f_cent in enumerate(note_frequencies):

      t_offset = n * sample_spacing
      t_length = total_samples - t_offset
      t = np.arange(t_length) / fs

      f_hz = fref * np.power(2, f_cent / octave_cents)
      
      waveform = sawtooth_harmonics(f_hz, t, power=2, random_phase=False)
      wave = lowpass_filter(waveform * adsr_curve[:t_length], fc=fc)

      data[t_offset:] += wave
    
      if chord and n < chord_notes:
         wave = lowpass_filter(waveform[:chord_length] * adsr_chord, fc=fc)
         data[-chord_length:] += wave

  return data


def compress(x):
    
    peak = np.abs(x).max()
    
    if peak > 1:
        return x / peak
    else:
        return x
    

## Widget

In [5]:


def parse_options(options):
    return [(o.name, n) for n, o in enumerate(options)]

head_layout = {'height': 'auto', 'width': 'auto'}
output_layout = {'align_items': 'center', 'align_content': 'center',}

slider_box = VBox([], layout=Layout(width='auto'))

cent_margin = 10

def clip_value(change):
    owner = change["owner"]
    note_sliders = slider_box.children
    if not owner.disabled:
        num_ns = len(note_sliders)
        i_ns = note_sliders.index(owner)
        
        low_ns = note_sliders[(i_ns - 1) % num_ns]
        high_ns = note_sliders[(i_ns + 1) % num_ns]

        low_value = low_ns.value + cent_margin
        high_value = high_ns.value - cent_margin

        if owner is note_sliders[0]:
            low_value -= octave_cents
        elif owner is note_sliders[-1]:
            high_value += octave_cents

        if owner.value < low_value:
            owner.value = low_value
        elif owner.value > high_value:
            owner.value = high_value

    

def notes2theta(note_list):
    theta = np.array(note_list) * 2 * np.pi / octave_cents
    
    return theta


plot_out = Output(layout=Layout(**output_layout))


def init_plot():
    with plot_out:
        
        plt.close(2)

        fig = plt.figure(2)
        fig.canvas.header_visible = False

        axis = fig.add_subplot(1, 1, 1, projection='polar')

        axis.set_theta_direction(-1)
        axis.set_theta_zero_location('N')

        axis.spines['polar'].set_visible(False)

        axis.set_yticks([1])
        axis.set_yticklabels([])
        axis.set_ylim(0, 1.1)
        plt.grid(False, axis='x')

        axis.fifth_order = None

        pline, = axis.plot(1, 1, marker="o", mfc="k", mec="k")

        pline.r = [1]
    
    return pline, axis


def set_notes(notes):
    pline, axis = init_plot()
    theta = notes2theta(notes)
    pline.set_data(theta[btn_scale.fifth_order], btn_notes.r)
    axis.set_xticks(theta)
    axis.set_xticklabels(btn_scale.xticklabels)


audio_out = Output(layout=Layout(**output_layout))
audio_out.f_root = a3_hz
audio_out.fs = sampling_rate
audio_out.player = None


def plot_notes(change):
    if type(change) is tuple or not change['owner'].disabled:
        with plot_out:
            
            if type(change) is tuple:
                notes = [nt.cents for nt in change]
            else:
                notes = [ns.value for ns in sliders_pool[:btn_notes.value]]
            
            set_notes(notes)
            plot_out.clear_output(wait=True)
            plt.tight_layout()
            plt.show()


@debounce(.2)
def generate_notes(change):
    if type(change) is tuple or not change['owner'].disabled:          
        if type(change) is tuple:
            freqs = [nt.cents for nt in change]
        else:
            freqs = [ns.value for ns in sliders_pool[:btn_notes.value]]
            
        freqs.append(freqs[0] + octave_cents)
        audio_data = synthesize(freqs, fref=audio_out.f_root, fs=audio_out.fs)
        audio_data = compress(audio_data)
        
        audio_kwargs = {"rate": audio_out.fs}
        if not in_colab:
            audio_kwargs["normalize"] = False

        if audio_out.player is None:
            audio = Audio(audio_data, autoplay=False, **audio_kwargs)  
            audio_out.player = audio
        else:
            audio_bytes = audio_out.player._make_wav(audio_data, **audio_kwargs)
            audio_out.player.data = audio_bytes
            audio_out.data = audio_data
        with audio_out:
            audio_out.clear_output(wait=True)
            display(audio_out.player)
        
    
def tune(change=None):

    n_notes = btn_notes.value
    
    temperament = btn_temperament.temperament[btn_temperament.value]
    
    if temperament.name == edo_name or n_notes > num_halftones:
        scale_notes = n_notes
    else:
        scale_notes = scales[n_notes][btn_scale.value].notes
    
    
    default_notes, f_root = get_notes(temperament, scale_notes)
    audio_out.f_root = f_root
    
    btn_scale.cents_root = get_cents(f_root)
    
    default_names = [dn.name for dn in default_notes]
    
    if 'F' in default_names:
        btn_scale.fifth_order = fifth_order(n_notes, first=default_names.index('F'))
    else:
        btn_scale.fifth_order = fifth_order(n_notes)
        
    
    if n_notes > num_halftones:
        btn_scale.xticklabels = []
    else:
        btn_scale.xticklabels = [dn.name for dn in default_notes]
    
    for ns in sliders_pool:
        ns.disabled = True
        
    if n_notes <= num_halftones:
        for ns, nt in zip(sliders_pool, default_notes):
            ns.description = nt.name
            ns.value = nt.cents
            ns.disabled = False
        
    plot_notes(default_notes)
    generate_notes(default_notes)
    
    
def update_name(change):
    
    if not btn_temperament.edo:
    
        slider = change["owner"]
        cents = change["new"]
        name = get_note_name(cents, btn_scale.cents_root)

        if name != slider.description:
            slider.description = name
            btn_scale.xticklabels[slider.index] = name

    
sliders_pool = ()
for n in range(num_halftones):
    new_slider = IntSlider(min=0, max=octave_cents, step=1, disabled=True,
                           continuous_update=continuous_update,
                           layout=Layout(width='auto'))
    
    new_slider.index = n
    
    new_slider.observe(clip_value, 'value')
    new_slider.observe(update_name, 'value')
    new_slider.observe(plot_notes, 'value')
    new_slider.observe(generate_notes, 'value')
    sliders_pool += new_slider,

    
def select_n_notes(change):
    
    n_notes = change["new"]

    
    btn_notes.r = np.ones(n_notes + 1)
    
    if n_notes > num_halftones:
        btn_temperament.temperament = temp_microtonal
        btn_temperament.options = parse_options(temp_microtonal)
        ui.right_sidebar = None
    else:
        btn_temperament.temperament = temp_regular
        btn_temperament.options = parse_options(temp_regular)
        btn_scale.disabled = False
        slider_box.children = ()
    
    
    btn_scale.options = parse_options(scales[n_notes])
    
    btn_scale.value = 0
    btn_temperament.value  = 0
    
    tune()
    
    if n_notes <= num_halftones:
        slider_box.children = sliders_pool[:n_notes]
        ui.right_sidebar= slider_box
    
    
def select_temperament(change):
    new_temp = btn_temperament.temperament[change["new"]]
    edo_selected = new_temp.name == edo_name
    btn_scale.disabled = edo_selected
    btn_temperament.edo = edo_selected
    tune(change)


btn_notes = Dropdown(options=valid_num_notes, value=default_num_notes,
                     description="Nr. of Notes", disabled=False,
                     layout=Layout(**head_layout))

btn_temperament = Dropdown(options=parse_options(temp_regular),
                           value=0, description="Temperament",
                           disabled=False, layout=Layout(**head_layout))

btn_temperament.edo = False

btn_temperament.temperament = temp_regular

btn_scale = Dropdown(options=parse_options(scales[default_num_notes]),
                     value=0, description="Scale/Mode",
                     disabled=False, layout=Layout(**head_layout))

btn_notes.observe(select_n_notes, 'value')
btn_temperament.observe(select_temperament, 'value')
btn_scale.observe(tune, 'value')

btn_reset = Button(description="Reset", layout=Layout(**head_layout))


btn_reset.on_click(tune)

ui = AppLayout(pane_widths=[2, 0, 3])
ui.right_sidebar = slider_box
ui.left_sidebar = VBox([btn_notes, btn_temperament, btn_scale, btn_reset, audio_out],
                  layout=Layout(justify_content='flex-start'))


select_n_notes({"old": 0, "new": default_num_notes})

display(ui, plot_out)

AppLayout(children=(VBox(children=(Dropdown(description='Nr. of Notes', index=5, layout=Layout(height='auto', …

Output(layout=Layout(align_content='center', align_items='center'))